In [20]:
import base64
import os

from dotenv import load_dotenv

from pydantic import BaseModel
from openai import OpenAI

load_dotenv()

openai_key = os.getenv('OPENAI_API_KEY')
openai_client = OpenAI()

class RequestBody(BaseModel):
    image: str

class MenuItemScored(BaseModel):
    name: str
    score: int
    red_flags: list[str]

class ResponseBody(BaseModel):
    item_list: list[MenuItemScored]

class MenuItem(BaseModel):
    name: str
    ingredients: list[str]

class Menu(BaseModel):
    choices: list[MenuItem]


ChatCompletion(id='chatcmpl-A7YSwDWP21zlgXNRBCQfRKDUdO5UZ', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='```json\n{\n  "choices": [\n    {\n      "name": "Fatto Tiramisu",\n      "ingredients": [\n        "Coffee liqueur-soaked sponge",\n        "mascarpone",\n        "chocolate"\n      ]\n    },\n    {\n      "name": "Scugnizzielli Nutella & Gelato",\n      "ingredients": [\n        "Fried mini pizza doughnuts",\n        "Nutella",\n        "vanilla gelato"\n      ]\n    },\n    {\n      "name": "Affogato",\n      "ingredients": [\n        "Vanilla gelato",\n        "espresso"\n      ]\n    },\n    {\n      "name": "Affogato Limoncello",\n      "ingredients": [\n        "Lemon sorbet",\n        "limoncello"\n      ]\n    },\n    {\n      "name": "Lemon Meringue",\n      "ingredients": [\n        "Vanilla biscuit",\n        "vanilla gelato",\n        "lemon curd",\n        "meringue"\n      ]\n    },\n    {\n      "name": "Chocola

In [ ]:
def calculate_score(request: RequestBody):    
    completion = openai_client.chat.completions.create(
        model="gpt-4o",
        max_tokens=4096,
        temperature=0.0,
        messages=[
            {
                "role": "system",
                "content": f"""
                You are a culinary expert designed to extract food items and their ingredients from a menu.
                Please identify every menu item the image and provide a list of every ingredient listed on the menu for that item.
                If the ingredients are not listed, make an educated guess about what the ingredients are. Simplify the ingredients to raw materials.
                You must return a correctly formatted JSON object with the menu item name and the list of ingredients in the image and follows the schema below:

                JSON Schema:
                {Menu.model_json_schema()}""",
            },
            {
                "role": "user",
                "content": [
                    {
                        "type": "text",
                        "text": """Extract the products from the image,
                        and describe them in a query in JSON format""",
                    },
                    {
                        "type": "image_url", "image_url": {
                        "url": f"data:image/jpeg;base64,{request.image}"}},

                ],
            },
            {
                "role": "assistant",
                "content": "```json",
            },
        ],
    )
    print(completion)

    return completion


In [ ]:
with open ("test_image.jpg", "rb") as image:
    image_data = base64.b64encode(image.read()).decode('utf-8')
    completion_message = calculate_score(RequestBody(image=image_data))

In [21]:
completion_message

ChatCompletion(id='chatcmpl-A7YSwDWP21zlgXNRBCQfRKDUdO5UZ', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='```json\n{\n  "choices": [\n    {\n      "name": "Fatto Tiramisu",\n      "ingredients": [\n        "Coffee liqueur-soaked sponge",\n        "mascarpone",\n        "chocolate"\n      ]\n    },\n    {\n      "name": "Scugnizzielli Nutella & Gelato",\n      "ingredients": [\n        "Fried mini pizza doughnuts",\n        "Nutella",\n        "vanilla gelato"\n      ]\n    },\n    {\n      "name": "Affogato",\n      "ingredients": [\n        "Vanilla gelato",\n        "espresso"\n      ]\n    },\n    {\n      "name": "Affogato Limoncello",\n      "ingredients": [\n        "Lemon sorbet",\n        "limoncello"\n      ]\n    },\n    {\n      "name": "Lemon Meringue",\n      "ingredients": [\n        "Vanilla biscuit",\n        "vanilla gelato",\n        "lemon curd",\n        "meringue"\n      ]\n    },\n    {\n      "name": "Chocola

In [38]:
import json

message_content = completion_message.choices[0].message.content
cleaned_content = message_content.replace('```json', '').replace('```', '').strip()
menu_dict = json.loads(cleaned_content)
menu = Menu(**menu_dict)
menu


Menu(choices=[MenuItem(name='Fatto Tiramisu', ingredients=['Coffee liqueur-soaked sponge', 'mascarpone', 'chocolate']), MenuItem(name='Scugnizzielli Nutella & Gelato', ingredients=['Fried mini pizza doughnuts', 'Nutella', 'vanilla gelato']), MenuItem(name='Affogato', ingredients=['Vanilla gelato', 'espresso']), MenuItem(name='Affogato Limoncello', ingredients=['Lemon sorbet', 'limoncello']), MenuItem(name='Lemon Meringue', ingredients=['Vanilla biscuit', 'vanilla gelato', 'lemon curd', 'meringue']), MenuItem(name='Chocolate Salted Caramel', ingredients=['Chocolate dipped biscuit', 'salted caramel gelato']), MenuItem(name='Chocolate Orange', ingredients=['Chocolate biscuit', 'chocolate orange gelato']), MenuItem(name='Limoncello', ingredients=['Limoncello']), MenuItem(name='Crema di Pistachio', ingredients=['Crema di Pistachio']), MenuItem(name='Amaro del Capo', ingredients=['Amaro del Capo']), MenuItem(name='Meloncello', ingredients=['Meloncello']), MenuItem(name='Espresso', ingredient

In [90]:
from langchain_core.messages import HumanMessage, SystemMessage
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_openai import ChatOpenAI

class CarbonCost(BaseModel):
    carbon_cost: int
    explanation: str
    red_flags: list[str]
    score: int

open_langchain = ChatOpenAI(model="gpt-4o", temperature=0)

def calculate_carbon_cost(ingredient: str):
    system_prompt = f"""
    You are a environmental expert designed to calculate the carbon cost of food items and explain the carbon cost of the food item in pounds.
    Calculate the carbon cost of the food item in the message below and explain why the carbon cost is that value.
    Also provide a list of red flags that indicate that the production of the food item involves unsustainable practices such as human rights violations in third world countries, animal welfare concerns, etc.
    Finally, based on this information, provide a sustainability score for the food item on a scale of 1 to 10
    """

    chat_history = ChatMessageHistory()
    chat_history.add_message(SystemMessage(content=system_prompt))
    chat_history.add_message(HumanMessage(content=ingredient))

    prompt_template = ChatPromptTemplate.from_messages(
        [MessagesPlaceholder("messages"), SystemMessage(content=system_prompt)])
    runnable = prompt_template | open_langchain.with_structured_output(
        schema=CarbonCost, include_raw=True)
    
    carbon_cost = runnable.invoke({"messages": chat_history.messages})

    return carbon_cost


In [91]:
cost = calculate_carbon_cost(menu.choices[0].name)


In [95]:
cost['parsed']

CarbonCost(carbon_cost=5, explanation='Tiramisu is a dessert that typically includes ingredients such as mascarpone cheese, eggs, sugar, coffee, cocoa, and ladyfingers. The carbon cost is influenced by the dairy products (mascarpone cheese and eggs), which have a higher carbon footprint due to methane emissions from cows and the energy-intensive processes involved in dairy farming. Additionally, coffee and cocoa production often involve long transportation routes and can be associated with deforestation and other environmental impacts.', red_flags=['Dairy production involves high methane emissions and energy use.', 'Coffee and cocoa production can be linked to deforestation and loss of biodiversity.', 'Potential human rights violations in coffee and cocoa farming, including poor working conditions and low wages.', 'Animal welfare concerns related to dairy farming.'], score=4)